In [1]:
import numpy as np
import pandas as pd
import gc

import scanpy as sc
import liana as li
import muon as mu

from benchmark import _benchmark, _sample_anndata, _sample_resource, _generate_counts

/home/dbdimitrov/anaconda3/envs/mofatalk/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
def _multiview_workflow(adata, resource):
    li.mt.cellphonedb.by_sample(adata, resource=resource,
                                groupby='cell_type', 
                                sample_key='sample',
                                n_perms=None, 
                                use_raw=False,
                                verbose=True)
    
    mdata = li.multi.lrs_to_views(adata,
                                  score_key='lr_means',
                                  sample_key = 'sample',
                                  obs_keys=None,
                                  lr_prop = 0.3,
                                  lrs_per_sample = 20,
                                  lrs_per_view = 20,
                                  samples_per_view = 3,
                                  min_variance = 0,
                                  lr_fill = 0,
                                  verbose=True
                                  )
    
    mu.tl.mofa(mdata,
           use_obs='union',
           convergence_mode='medium',
           outfile='models/mofatalk.h5ad',
           n_factors=10,
           verbose=False
           )
    
    print(len(mdata.mod.keys()))

In [3]:
sparsity = 0.9
n_samples = 20
view_range = [4, 8, 12, 16, 20]
n_times = 5
output_file = 'factor_stats.csv'

In [4]:
benchmark_stats = pd.DataFrame(columns=["n_views", "time", "memory"])

for n_views in view_range:
    print(n_views)
    adata = _sample_anndata(sparsity=sparsity, n_ct=n_views, n_vars=2000, n_obs=3000, seed=1337)
    resource = _sample_resource(adata=adata, n_lrs=2000)

    adatas = []
    adatas.append(adata)
    rng = np.random.default_rng(1337)
    for sample in range(n_samples):
        adatas.append(adata.copy())
        adatas[-1].X = _generate_counts(rng=rng, n_vars=adata.shape[1], n_obs=adata.shape[0], sparsity=sparsity)
        adata.obs['sample'] = f"sample_{sample}"
    
    adata = sc.concat(adatas, join='outer')
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
        
    for _ in range(n_times):
        time, memory = _benchmark(function=_multiview_workflow,
                                  adata=adata,
                                  resource=resource,
                                  )
        benchmark_stats.loc[len(benchmark_stats)] = [n_views**2, time, memory]
        benchmark_stats.to_csv(output_file, index=False)
        gc.collect()

4
 NNZ fraction: 0.09993416666666667


/home/dbdimitrov/anaconda3/envs/mofatalk/lib/python3.10/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Converting `sample` to categorical!
Now running: sample_0:   0%|          | 0/20 [00:00<?, ?it/s]/home/dbdimitrov/anaconda3/envs/mofatalk/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
/home/dbdimitrov/anaconda3/envs/mofatalk/lib/python3.10/site-packages/liana/method/_pipe_utils/_pre.py:151: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
Now running: sample_1:   5%|▌         | 1/20 [00:00<00:09,  2.08it/s]/home/dbdimitrov/anaconda3/envs/mofatalk/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWa


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Loaded view='CT0&CT0' group='group1' with N=20 samples and D=658 features...
Loaded view='CT2&CT0' group='group1' with N=20 samples and D=663 features...
Loaded view='CT2&CT1' group='group1' with N=20 samples and D=781 features...
Loaded view='CT2&CT3' group='group1' with N=20 samples and D=681 features...
Loaded view='CT2&CT2' group='group1' with N=20 samples and D=670 features

100%|██████████| 16/16 [00:00<00:00, 92.21it/s]



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Loaded view='CT0&CT0' group='group1' with N=20 samples and D=658 features...
Loaded view='CT2&CT0' group='group1' with N=20 samples and D=663 features...
Loaded view='CT2&CT1' group='group1' with N=20 samples and D=781 features...
Loaded view='CT2&CT3' group='group1' with N=20 samples and D=681 features...
Loaded view='CT2&CT2' group='group1' with N=20 samples and D=670 features

...


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Loaded view='CT12&CT3' group='group1' with N=20 samples and D=817 features...
Loaded view='CT15&CT3' group='group1' with N=20 samples and D=777 features...
Loaded view='CT9&CT4' group='group1' with N=20 samples and D=712 features...
Loaded view='CT4&CT7' group='group1' with N=20 samples and D=812 features...
Loaded view='CT7&CT16' group='group1' with N=20 samples and D=852 featu

100%|██████████| 400/400 [01:20<00:00,  4.96it/s]



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Loaded view='CT12&CT3' group='group1' with N=20 samples and D=817 features...
Loaded view='CT15&CT3' group='group1' with N=20 samples and D=777 features...
Loaded view='CT9&CT4' group='group1' with N=20 samples and D=712 features...
Loaded view='CT4&CT7' group='group1' with N=20 samples and D=812 features...
Loaded view='CT7&CT16' group='group1' with N=20 samples and D=852 featu

100%|██████████| 400/400 [01:11<00:00,  5.59it/s]



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Loaded view='CT12&CT3' group='group1' with N=20 samples and D=817 features...
Loaded view='CT15&CT3' group='group1' with N=20 samples and D=777 features...
Loaded view='CT9&CT4' group='group1' with N=20 samples and D=712 features...
Loaded view='CT4&CT7' group='group1' with N=20 samples and D=812 features...
Loaded view='CT7&CT16' group='group1' with N=20 samples and D=852 featu